In [1]:
# Load libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from joblib import dump
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim.lr_scheduler import LambdaLR

Yolo-Modeling

In [2]:
df = pd.read_csv('onetoohot.csv', low_memory=False)
bool_columns_list = [col for col in df if col != 'Amount']
# Define your features (X) and target (y) from the DataFrame
X = df[bool_columns_list]
y = df['Amount']

# Splitting the dataset into training and test sets (as you've done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Further split the training set into a new training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Converting to PyTorch tensors and moving to the specified device
device = torch.device('cuda')

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Creating datasets and dataloaders for training, validation, and test sets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=750, shuffle=True)
val_loader = DataLoader(val_data, batch_size=750, shuffle=False)
test_loader = DataLoader(test_data, batch_size=750, shuffle=False)


In [3]:
device = torch.device("cuda")
print(device)

cuda


In [4]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.dropout1 = nn.Dropout(0.2)  # First dropout layer
        self.fc2 = nn.Linear(1024, 256)
        self.dropout2 = nn.Dropout(0.2)  # Second dropout layer
        self.fc3 = nn.Linear(256, 128)
        self.dropout3 = nn.Dropout(0.2)  # Third dropout layer
        self.fc4 = nn.Linear(128, 64)
        self.dropout4 = nn.Dropout(0.2)  # Fourth dropout layer
        self.fc5 = nn.Linear(64, 32)
        self.dropout5 = nn.Dropout(0.2)  # Fifth dropout layer
        self.fc6 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.relu(self.fc5(x))
        x = self.dropout5(x)
        x = self.fc6(x)  # Apply the last linear layer
        return x

In [5]:
# Initialize the model, loss function, and optimizer
model = RegressionModel(X_train_tensor.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.005, betas=(0.9, 0.999), eps=1e-08)


# Define a lambda function for learning rate decay
def lr_lambda(epoch):
    initial_lr = 0.05
    final_lr = 0.001
    max_epochs = 20000
    decay_rate = (final_lr / initial_lr) ** (1 / max_epochs)
    return decay_rate ** epoch
scheduler = LambdaLR(optimizer, lr_lambda)

In [8]:
# Training the model
# Optimizer: Adam with specified parameters and a learning rate of 0.001

n_epochs = 20000
early_stop_threshold = 0.85
for epoch in range(n_epochs):
    
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)  # Move data to the device
        
        # Clear the gradients of all optimized tensors
        optimizer.zero_grad()
        
        # Forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # Calculate the loss
        loss = criterion(output.view(-1), target)
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Perform a single optimization step (parameter update)
        optimizer.step()
        scheduler.step()
        # Update running training loss
        train_loss += loss.item() * data.size(0)
    
    model.eval()
    with torch.no_grad():
        val_predictions = []
        val_targets = []
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            val_predictions.extend(outputs.view(-1).cpu().numpy())
            val_targets.extend(y.cpu().numpy())
        
        val_r2_score = r2_score(val_targets, val_predictions)
        print(f'Epoch {epoch+1}/{n_epochs}, Validation R2 Score: {val_r2_score}')

        if val_r2_score >= early_stop_threshold:
            print(f"Early stopping at epoch {epoch+1} with R2 Score: {val_r2_score}")
            break

Epoch 1/20000, Validation R2 Score: 0.4162722125085949
Epoch 2/20000, Validation R2 Score: 0.4169566072594558
Epoch 3/20000, Validation R2 Score: 0.4112554875617893
Epoch 4/20000, Validation R2 Score: 0.4211707273037957
Epoch 5/20000, Validation R2 Score: 0.3417786956773181
Epoch 6/20000, Validation R2 Score: 0.39125387340528983
Epoch 7/20000, Validation R2 Score: 0.4153126541596779
Epoch 8/20000, Validation R2 Score: 0.4166197253334656
Epoch 9/20000, Validation R2 Score: 0.4206587765682882
Epoch 10/20000, Validation R2 Score: 0.4047123863254881
Epoch 11/20000, Validation R2 Score: 0.41526349047334576
Epoch 12/20000, Validation R2 Score: 0.4174787497178817
Epoch 13/20000, Validation R2 Score: 0.4157603558401336
Epoch 14/20000, Validation R2 Score: 0.4178495888470194
Epoch 15/20000, Validation R2 Score: 0.4165321716687391
Epoch 16/20000, Validation R2 Score: 0.4157339797179539
Epoch 17/20000, Validation R2 Score: 0.4101346480572833
Epoch 18/20000, Validation R2 Score: 0.4143593095339074

KeyboardInterrupt: 

In [7]:
# Evaluating the model in detail
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).view(-1)
    
    # Convert tensors to numpy arrays for compatibility with sklearn metrics
    y_test_np = y_test_tensor.cpu().numpy()
    predictions_np = predictions.cpu().numpy()

    # Calculating MAE
    mae = mean_absolute_error(y_test_np, predictions_np)
    print(f'MAE: {mae}')

    # RMSE (already calculated)
    mse = criterion(predictions, y_test_tensor)
    rmse = torch.sqrt(mse)
    print(f'RMSE: {rmse.item()}')

    # R2 Score (already calculated)
    r2 = r2_score(y_test_np, predictions_np)
    print(f'R2 Score: {r2}')

    # Calculating MAPE
    mape = np.mean(np.abs((y_test_np - predictions_np) / y_test_np)) * 100
    print(f'MAPE: {mape}')

    # Adjusted R2 (for multiple regression models)
    n = len(y_test_np) # Number of samples
    p = X_test_tensor.shape[1] # Number of independent variables
    adjusted_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    print(f'Adjusted R2 Score: {adjusted_r2}')

    print("---------First 10 Predictions---------")
    for actual, predicted in zip(y_test_np[:10], predictions_np[:10]):
        absolute_error = np.abs(actual - predicted)
        relative_error = (absolute_error / actual) * 100 if actual != 0 else float('inf')
        print(f"Actual/Predicted: {actual}/{predicted}")
        print(f"Abs Error: {absolute_error}")
        print(f"Rel Error: {relative_error:.2f}%")

MAE: 142.04791259765625
RMSE: 206.29165649414062
R2 Score: 0.4258465895680672
MAPE: inf
Adjusted R2 Score: 0.42048319010729496
---------First 10 Predictions---------
Actual/Predicted: 599.0/706.3789672851562
Abs Error: 107.37896728515625
Rel Error: 17.93%
Actual/Predicted: 435.0/433.2632751464844
Abs Error: 1.736724853515625
Rel Error: 0.40%
Actual/Predicted: 1523.0/1104.230712890625
Abs Error: 418.769287109375
Rel Error: 27.50%
Actual/Predicted: 648.5614624023438/652.218994140625
Abs Error: 3.65753173828125
Rel Error: 0.56%
Actual/Predicted: 845.0/1000.3237915039062
Abs Error: 155.32379150390625
Rel Error: 18.38%
Actual/Predicted: 376.0/461.2226257324219
Abs Error: 85.22262573242188
Rel Error: 22.67%
Actual/Predicted: 416.0/405.9302978515625
Abs Error: 10.0697021484375
Rel Error: 2.42%
Actual/Predicted: 899.0/726.4105224609375
Abs Error: 172.5894775390625
Rel Error: 19.20%
Actual/Predicted: 648.5614624023438/618.3818359375
Abs Error: 30.17962646484375
Rel Error: 4.65%
Actual/Predicted

C:\Users\Tono\AppData\Local\Temp\ipykernel_22660\3384580297.py:26: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_test_np - predictions_np) / y_test_np)) * 100


In [ ]:
# save model
#torch.save(model, 'Amazon.pth')